In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
data=pd.read_csv('medicine.csv')

In [3]:
data.head()

,index,Drug_Name,Reason,Description
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,Acne,Mild to moderate acne (spots)
1,2,A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA Ret 0...,Acne,A RET 0.025% is a prescription medicine that i...
2,3,ACGEL CL NANO Gel 15gm,Acne,It is used to treat acne vulgaris in people 12...
3,4,ACGEL NANO Gel 15gm,Acne,It is used to treat acne vulgaris in people 12...
4,5,Acleen 1% Lotion 25ml,Acne,treat the most severe form of acne (nodular ac...


In [4]:
data.isnull().sum()

index          0
Drug_Name      0
Reason         0
Description    0
dtype: int64

In [5]:
data.shape

(9720, 4)

In [6]:
data.describe()

,index
count,9720.000000
mean,4860.500000
std,2806.066642
min,1.000000
25%,2430.750000
50%,4860.500000
75%,7290.250000
max,9720.000000


In [7]:
data['Description']

0                           Mild to moderate acne (spots)
1       A RET 0.025% is a prescription medicine that i...
2       It is used to treat acne vulgaris in people 12...
3       It is used to treat acne vulgaris in people 12...
4       treat the most severe form of acne (nodular ac...
                              ...                        
9715                              used for treating warts
9716                        used to soften the skin cells
9717                                       used for scars
9718                                      used for wounds
9719    used to treat and remove raised warts (usually...
Name: Description, Length: 9720, dtype: object

In [8]:
data['Description']=data['Description'].apply(lambda x: x.split())
data['Reason']=data['Reason'].apply(lambda x: x.split())

In [9]:
data['Description']=data['Description'].apply(lambda x: [i.replace(" ","") for i in x] )

In [10]:

data['Tags']=data['Description']+data['Reason']

In [11]:
new_data=data[['index','Drug_Name','Tags']]

In [12]:
new_data.head()

,index,Drug_Name,Tags
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,"[Mild, to, moderate, acne, (spots), Acne]"
1,2,A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA Ret 0...,"[A, RET, 0.025%, is, a, prescription, medicine..."
2,3,ACGEL CL NANO Gel 15gm,"[It, is, used, to, treat, acne, vulgaris, in, ..."
3,4,ACGEL NANO Gel 15gm,"[It, is, used, to, treat, acne, vulgaris, in, ..."
4,5,Acleen 1% Lotion 25ml,"[treat, the, most, severe, form, of, acne, (no..."


In [13]:
new_data['Tags']=new_data['Tags'].apply(lambda x: ' '.join(x))

C:\Users\DELL\AppData\Local\Temp\ipykernel_20324\794317305.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Tags']=new_data['Tags'].apply(lambda x: ' '.join(x))


In [14]:
new_data['Tags']=new_data['Tags'].apply(lambda x:x.lower())



C:\Users\DELL\AppData\Local\Temp\ipykernel_20324\3184014922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Tags']=new_data['Tags'].apply(lambda x:x.lower())


In [15]:
!pip install nltk


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\DELL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [16]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')


In [17]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return ' '.join(y)

In [18]:
new_data['Tags']=new_data['Tags'].apply(stem)

C:\Users\DELL\AppData\Local\Temp\ipykernel_20324\1233099800.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Tags']=new_data['Tags'].apply(stem)


In [19]:
vectors=cv.fit_transform(new_data['Tags']).toarray()

In [20]:
vectors.shape

(9720, 806)

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)

In [22]:
def recommendation(med):
    med_index=new_data[new_data['Drug_Name']==med].index[0]
    distance=similarity[med_index]
    med_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
    for i in med_list:
        print(new_data.iloc[i[0]].Drug_Name)

In [23]:
import pickle
pickle.dump(new_data, open('med_dict.pkl','wb'))
pickle.dump(similarity, open('similarity.pkl','wb'))
import streamlit as st


In [24]:
import joblib
import gzip

with open("models/med_dict.pkl", "rb") as f_in:
    with gzip.open("models/med_dict.pkl.gz", "wb") as f_out:
        f_out.writelines(f_in)

with open("models/similarity.pkl", "rb") as f_in:
    with gzip.open("models/similarity.pkl.gz", "wb") as f_out:
        f_out.writelines(f_in)